# from Model1

In [ ]:
import cv2
import numpy as np
from keras.models import load_model
from statistics import mode
from utils.datasets import get_labels
from utils.inference import detect_faces
from utils.inference import draw_text
from utils.inference import draw_bounding_box
from utils.inference import apply_offsets
from utils.inference import load_detection_model
from utils.preprocessor import preprocess_input

USE_WEBCAM = True # If false, loads video file source

# parameters for loading data and images
emotion_model_path = './models/emotion_model.hdf5'
emotion_labels = get_labels('fer2013')

# hyper-parameters for bounding boxes shape
frame_window = 10
emotion_offsets = (20, 40)

# loading models
face_cascade = cv2.CascadeClassifier('./models/haarcascade_frontalface_default.xml')
emotion_classifier = load_model(emotion_model_path)

# getting input model shapes for inference
emotion_target_size = emotion_classifier.input_shape[1:3]

# starting lists for calculating modes
emotion_window = []

# starting video streaming

cv2.namedWindow('window_frame')
video_capture = cv2.VideoCapture(0)

# Select video or webcam feed
cap = None
if (USE_WEBCAM == True):
    cap = cv2.VideoCapture(0) # Webcam source
else:
    cap = cv2.VideoCapture('./inp.mp4') # Video file source

    #out = cv2.VideoWriter('output.avi', -1, 60.0, (640,480))

while cap.isOpened():
    video_capture.set(cv2.CAP_PROP_FPS, 10)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    print("fps:", fps)
    ret, bgr_image = cap.read()
    bgr_image = cv2.resize(bgr_image,(640,360))

    #bgr_image = video_capture.read()[1]
    #out.write(bgr_image)

    gray_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2GRAY)
    rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)

    faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5,
			minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

    for face_coordinates in faces:

        x1, x2, y1, y2 = apply_offsets(face_coordinates, emotion_offsets)
        gray_face = gray_image[y1:y2, x1:x2]
        try:
            gray_face = cv2.resize(gray_face, (emotion_target_size))
        except:
            continue

        gray_face = preprocess_input(gray_face, True)
        gray_face = np.expand_dims(gray_face, 0)
        gray_face = np.expand_dims(gray_face, -1)
        emotion_prediction = emotion_classifier.predict(gray_face)
        emotion_probability = np.max(emotion_prediction)
        emotion_label_arg = np.argmax(emotion_prediction)
        emotion_text = emotion_labels[emotion_label_arg]
        emotion_window.append(emotion_text)

        if len(emotion_window) > frame_window:
            emotion_window.pop(0)
        try:
            emotion_mode = mode(emotion_window)
        except:
            continue

        if emotion_text == 'angry':
            color = emotion_probability * np.asarray((255, 0, 0))
        elif emotion_text == 'sad':
            color = emotion_probability * np.asarray((0, 0, 255))
        elif emotion_text == 'happy':
            color = emotion_probability * np.asarray((255, 255, 0))
        elif emotion_text == 'surprise':
            color = emotion_probability * np.asarray((0, 255, 255))
        else:
            color = emotion_probability * np.asarray((0, 255, 0))

        color = color.astype(int)
        color = color.tolist()

        draw_bounding_box(face_coordinates, rgb_image, color)
        draw_text(face_coordinates, rgb_image, emotion_mode,
                  color, 0, -45, 1, 1)

    bgr_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)



    cv2.imshow('window_frame', bgr_image)


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    

cap.release()

#out.release()

cv2.destroyAllWindows()


# from Model2

In [3]:
import numpy as np
import cv2
from keras.preprocessing import image
import time

#-----------------------------
#opencv initialization

face_cascade = cv2.CascadeClassifier('./models/haarcascade_frontalface_default.xml')

#-----------------------------
#face expression recognizer initialization
from keras.models import model_from_json
model = model_from_json(open("./models/facial_expression_model_structure.json", "r").read())
model.load_weights('./models/facial_expression_model_weights.h5') #load weights
#-----------------------------

emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
#cv2.namedWindow('window_frame')
#video_capture = cv2.VideoCapture(0)

# Select video or webcam feed
#cap = None
#cap = cv2.VideoCapture("./inp.mp4") #process videos
cap = cv2.VideoCapture(0) #process real time web-cam

frame = 0


while cap.isOpened():
	
	ret, img = cap.read()
	
	img = cv2.resize(img, (640, 360))
	img = img[0:308,:]

	gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
	#print("there1")
	faces = face_cascade.detectMultiScale(gray, 1.3, 5)
	#print("there2")
	for (x,y,w,h) in faces:
		if (True): #trick: ignore small faces
			cv2.rectangle(img,(x,y),(x+w,y+h),(64,64,64),2) #highlight detected face
			#print("there3")
			detected_face = img[int(y):int(y+h), int(x):int(x+w)] #crop detected face
			detected_face = cv2.cvtColor(detected_face, cv2.COLOR_BGR2GRAY) #transform to gray scale
			detected_face = cv2.resize(detected_face, (48, 48)) #resize to 48x48
			
			img_pixels = image.img_to_array(detected_face)
			img_pixels = np.expand_dims(img_pixels, axis = 0)
			
			img_pixels /= 255 #pixels are in scale of [0, 255]. normalize all pixels in scale of [0, 1]
			
			#------------------------------
			
			predictions = model.predict(img_pixels) #store probabilities of 7 expressions
			max_index = np.argmax(predictions[0])
			
			#background of expression list
			overlay = img.copy()
			opacity = 0.4
			cv2.rectangle(img,(x+w+10,y-25),(x+w+150,y+115),(64,64,64),cv2.FILLED)
			cv2.addWeighted(overlay, opacity, img, 1 - opacity, 0, img)
			
			#connect face and expressions
			cv2.line(img,(int((x+x+w)/2),y+15),(x+w,y-20),(255,255,255),1)
			cv2.line(img,(x+w,y-20),(x+w+10,y-20),(255,255,255),1)
			
			emotion = ""
			for i in range(len(predictions[0])):
				emotion = "%s %s%s" % (emotions[i], round(predictions[0][i]*100, 2), '%')
				
				"""if i != max_index:
					color = (255,0,0)"""
					
				color = (255,255,255)
				
				cv2.putText(img, emotion, (int(x+w+15), int(y-12+i*20)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
				
			#-------------------------
	
	cv2.imshow('img',img)
	
	frame = frame + 1
	#print(frame)
	
	#---------------------------------
	
	#if frame > 227:
	#	break
	
	if cv2.waitKey(1) & 0xFF == ord('q'): #press q to quit
		break

#kill open cv things
cap.release()
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2
from keras.preprocessing import image
import time

#-----------------------------
#opencv initialization

face_cascade = cv2.CascadeClassifier('./models/haarcascade_frontalface_default.xml')

#-----------------------------
#face expression recognizer initialization
from keras.models import model_from_json
model = model_from_json(open("./models/facial_expression_model_structure.json", "r").read())
model.load_weights('./models/facial_expression_model_weights.h5') #load weights
#-----------------------------

emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
#cv2.namedWindow('window_frame')
#video_capture = cv2.VideoCapture(0)

# Select video or webcam feed
#cap = None
#cap = cv2.VideoCapture("./inp.mp4") #process videos
cap = cv2.VideoCapture(0) #process real time web-cam

frame = 0


while cap.isOpened():
	
	ret, img = cap.read()
	
	img = cv2.resize(img, (640, 360))
	img = img[0:708,:]

	gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
	#print("there1")
	faces = face_cascade.detectMultiScale(gray, 1.3, 5)
	#print("there2")
	for (x,y,w,h) in faces:
		if (True): #trick: ignore small faces
			cv2.rectangle(img,(x,y),(x+w,y+h),(64,64,64),2) #highlight detected face
			#print("there3")
			detected_face = img[int(y):int(y+h), int(x):int(x+w)] #crop detected face
			detected_face = cv2.cvtColor(detected_face, cv2.COLOR_BGR2GRAY) #transform to gray scale
			detected_face = cv2.resize(detected_face, (48, 48)) #resize to 48x48
			
			img_pixels = image.img_to_array(detected_face)
			img_pixels = np.expand_dims(img_pixels, axis = 0)
			
			img_pixels /= 255 #pixels are in scale of [0, 255]. normalize all pixels in scale of [0, 1]
			
			#------------------------------
			
			predictions = model.predict(img_pixels) #store probabilities of 7 expressions
			max_index = np.argmax(predictions[0])
			
			#background of expression list
			#overlay = img.copy()
			#opacity = 0.4
			#cv2.rectangle(img,(200,200),(200,200),(64,64,64),cv2.FILLED)
			#cv2.addWeighted(overlay, opacity, img, 1 - opacity, 0, img)
			
			#connect face and expressions
			cv2.line(img,(int((x+x+w)/2),y+15),(x+w,y-20),(255,255,255),1)
			cv2.line(img,(x+w,y-20),(x+w+10,y-20),(255,255,255),1)
			emotions_dict = {0:'angry', 1:'disgust', 2:'fear', 3:'happy', 4:'sad', 5:'surprise', 6:'neutral'}
			color_dict = {0:(255,0,0), 1:(255,150,0), 2:(0,255,0), 3:(0,255,255), 4:(0,0,255), 5:(255,0,255), 6:(255,255,0)}            
			cv2.putText(img, emotions_dict[max_index], (int(x+w+15), int(y-15)), cv2.FONT_HERSHEY_TRIPLEX, 1,color_dict[max_index], 1)
			#emotion = ""
			#for i in range(len(predictions[0])):
				#th = round(predictions[0][i]*100, 2)
				#emotion = "%s %s%s" % (emotions[i], th, '%')
				
				
				#print(max_index)
				#color = (255,255,255)
				
				#cv2.putText(img, emotion, (int(x+w+15), int(y-12+i*20)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
				
#-------------------------
	
	cv2.imshow('img',img)
	
	frame = frame + 1
	#print(frame)
	
	#---------------------------------
	
	#if frame > 227:
	#	break
	
	if cv2.waitKey(1) & 0xFF == ord('q'): #press q to quit
		break

#kill open cv things
cap.release()
cv2.destroyAllWindows()

Using TensorFlow backend.
